In [8]:
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder

In [4]:
df = pd.read_csv('../data/survey_data_before_pre_processing.csv')

### Remove Refused Persons

General rundown here is to calculate the amount of refusals per person as a percentage of total questions refused.

Find some cutoff and drop those people

### Convert all Refused to NaN

We will be imputing these, but need them to be nulls

### Ordinal Encode

There are a lot these.....

In [7]:
df.income_category.unique()

array(['$50,000 to $99,999', 'Less than $50,000', '$100,000 or more'],
      dtype=object)

In [9]:
cats = [['Greatest (Before 1928)','Silent (1928 - 1945)',
         'Baby Boomers (1946 - 1964)',
         'Generation X (1965 - 1980)',
         'Millennials (1981 - 1996)',
         'iGen\u200e/Gen Z (1997 - )'],
        ['Less than high school',
         'High school',
         'Some college',
         'Bachelor\'s degree or higher'],
        ['Less than $50,000',
         '$50,000 to $99,999',
         '$100,000 or more']]

o_enc = OrdinalEncoder(categories=cats)
o_enc.fit(df[['generation', 'educ_category', 'income_category']])
X_ord = o_enc.transform(df[['generation', 'educ_category', 'income_category']])
X_ord

array([[1., 1., 1.],
       [2., 0., 0.],
       [2., 3., 0.],
       ...,
       [4., 2., 2.],
       [3., 2., 0.],
       [4., 1., 0.]])

In [10]:
df[['generation', 'educ_category', 'income_category']]

,generation,educ_category,income_category
0,Silent (1928 - 1945),High school,"$50,000 to $99,999"
1,Baby Boomers (1946 - 1964),Less than high school,"Less than $50,000"
2,Baby Boomers (1946 - 1964),Bachelor's degree or higher,"Less than $50,000"
3,Silent (1928 - 1945),Bachelor's degree or higher,"$100,000 or more"
4,Millennials (1981 - 1996),Some college,"$50,000 to $99,999"
...,...,...,...
22290,Baby Boomers (1946 - 1964),Some college,"Less than $50,000"
22291,Baby Boomers (1946 - 1964),Less than high school,"$50,000 to $99,999"
22292,Millennials (1981 - 1996),Some college,"$100,000 or more"
22293,Generation X (1965 - 1980),Some college,"Less than $50,000"


### Train Test Split

### Iteratively Impute Nulls

Now all of our data is ordinal encoded or dummied, iteratively impute the nulls

### Standard Scale

Will let us explore knn or clustering if wanted